In [1]:
import pandas as pd
from plotly.offline import iplot
import plotly.graph_objs as go
import numpy as np
from sklearn import metrics
import math
import matplotlib.pyplot as plt
from collections import defaultdict
from functools import partial
from itertools import repeat
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import roc_curve, auc
from keras.utils import to_categorical
from keras import models
from keras import layers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Dropout, Activation
import keras.backend as K
import os
from keras import metrics

Using TensorFlow backend.


In [2]:
# returns dataframe
# params: full -> True = full dataset, False = first file

def grab_data(full, path):
    if full:
        return pd.concat([pd.read_csv(path+file, index_col=False) for file in os.listdir(path)])
    else:
        for file in os.listdir(path):
            if "10_1_" not in file: continue
            return pd.read_csv(path+file, index_col=False)

In [4]:
# import data
nBodies = 4
workDir = "/Users/brandonmanley/Documents/nBody/data"
dataPath = "/Users/brandonmanley/Documents/nBody/data/brutusSim/"+str(nBodies)+"body/"
        
df = grab_data(False, dataPath)
df.head()

,finalFile,eventID,m1,m2,m3,m4,x1,x2,x3,x4,...,yf3,yf4,dxf1,dxf2,dxf3,dxf4,dyf1,dyf2,dyf3,dyf4
0,1,10001,15.493888,35.570682,69.500624,74.990475,8.329506,6.776766,-4.461837,-5.624802,...,2.632498,-9.332945,0.921563,0.052930,0.346384,-0.790396,0.789834,-0.910954,0.445991,-0.566522
1,1,10002,15.493888,35.570682,69.500624,74.990475,8.329506,6.776766,-4.461837,-5.624802,...,2.634235,-9.335154,0.918182,0.050424,0.347452,-0.789499,0.793880,-0.912933,0.443628,-0.564230
2,1,10003,15.493888,35.570682,69.500624,74.990475,8.329506,6.776766,-4.461837,-5.624802,...,2.635963,-9.337353,0.914798,0.047920,0.348520,-0.788601,0.797931,-0.914913,0.441266,-0.561939
3,1,10004,15.493888,35.570682,69.500624,74.990475,8.329506,6.776766,-4.461837,-5.624802,...,2.637682,-9.339544,0.911411,0.045419,0.349586,-0.787703,0.801988,-0.916894,0.438906,-0.559649
4,1,10005,15.493888,35.570682,69.500624,74.990475,8.329506,6.776766,-4.461837,-5.624802,...,2.639392,-9.341725,0.908020,0.042920,0.350653,-0.786805,0.806052,-0.918877,0.436546,-0.557361


In [5]:
# INPUT: want mass, x/y pos, dx/dy for n bodies -> total input is n*5 + 1 (time)
# OUTPUT: want x/y, dx/dy -> total input is n*4


i_col, o_col = [], []
colNames = ["m", "x", "y", "dx", "dy"]

for col in colNames:
    for n in range(1, nBodies+1):
        i_col.append(col+str(n))
        if col != "m":
            o_col.append(col+"f"+str(n))
i_col.append("t")
    
X1 = df.as_matrix(columns=i_col)
y1 = df.as_matrix(columns=o_col)

print(X1.shape, y1.shape)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning:

Method .as_matrix will be removed in a future version. Use .values instead.

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning:

Method .as_matrix will be removed in a future version. Use .values instead.



(579662, 21) (579662, 16)


In [12]:
# make predictions
loss_function = 'mse'
my_metrics = ['mae']

network = models.Sequential()
network.add(layers.Dense(300, activation='relu', input_dim=X1.shape[1]))
for i in range(9):
    network.add(layers.Dense(300, activation='relu'))
network.add(layers.Dense(y1.shape[1],activation='linear'))
network.compile(optimizer='adam', loss=loss_function, metrics=my_metrics)
network.load_weights(workDir + '/weights/final_{0}body.h5'.format(nBodies))
yPred = network.predict(X1)

In [30]:
# save predictions
dfPred = pd.DataFrame(yPred, index=None, columns=[o+"p" for o in o_col])
for col in dfPred.columns:
    df[col] = dfPred[col]
df.to_csv(workDir+"/pred/comb10_1_{0}.csv".format(nBodies))

In [29]:
df

,finalFile,eventID,m1,m2,m3,m4,x1,x2,x3,x4,...,yf3p,yf4p,dxf1p,dxf2p,dxf3p,dxf4p,dyf1p,dyf2p,dyf3p,dyf4p
0,1,10001,15.493888,35.570682,69.500624,74.990475,8.329506,6.776766,-4.461837,-5.624802,...,2.799202,-9.366252,0.810941,1.013085,0.297682,-0.732393,0.171126,-0.075296,0.173353,-0.546479
1,1,10002,15.493888,35.570682,69.500624,74.990475,8.329506,6.776766,-4.461837,-5.624802,...,2.793642,-9.379079,0.831956,1.015389,0.293573,-0.742404,0.187425,-0.077188,0.172529,-0.553372
2,1,10003,15.493888,35.570682,69.500624,74.990475,8.329506,6.776766,-4.461837,-5.624802,...,2.788079,-9.391915,0.852983,1.017690,0.289462,-0.752418,0.203731,-0.079082,0.171704,-0.560268
3,1,10004,15.493888,35.570682,69.500624,74.990475,8.329506,6.776766,-4.461837,-5.624802,...,2.782376,-9.404632,0.870106,1.022435,0.286084,-0.761900,0.218509,-0.079991,0.171571,-0.567232
4,1,10005,15.493888,35.570682,69.500624,74.990475,8.329506,6.776766,-4.461837,-5.624802,...,2.776294,-9.417045,0.876863,1.033680,0.284650,-0.769975,0.229235,-0.078282,0.173276,-0.574382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579657,1,2812556,44.521552,97.763363,82.487713,59.878320,2.566150,7.873928,-8.220132,8.455770,...,0.145956,0.886424,-6.276968,3.961956,-0.490675,-0.485251,2.617706,-0.234029,-0.815854,1.448339
579658,1,2812557,44.521552,97.763363,82.487713,59.878320,2.566150,7.873928,-8.220132,8.455770,...,0.147165,0.885346,-6.279742,3.962605,-0.491474,-0.484032,2.618836,-0.234105,-0.815882,1.448984
579659,1,2812558,44.521552,97.763363,82.487713,59.878320,2.566150,7.873928,-8.220132,8.455770,...,0.148229,0.884208,-6.282435,3.963246,-0.492372,-0.482638,2.619947,-0.234094,-0.815827,1.449579
579660,1,2812559,44.521552,97.763363,82.487713,59.878320,2.566150,7.873928,-8.220132,8.455770,...,0.149294,0.883069,-6.285127,3.963886,-0.493270,-0.481244,2.621058,-0.234083,-0.815771,1.450174
